In [4]:
import pandas as pd
import numpy as np
import ast
import nltk
import pickle 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
movies = pd.read_csv("movies.csv")
credit = pd.read_csv("Creditscsv.csv",low_memory=False)

In [6]:
thesoldFreq = 9.0
removeColums = credit.columns[credit.isnull().sum() > thesoldFreq]
credit = credit.drop(removeColums,axis=1)

# we merged the data
-> Don't use unnessary terms for merging the data: like tuple(2,3) they does't merge

# merging col
-> geners
-> id
-> keywords
->title
->overview
-> cast
-> crew

In [7]:
movie = movies.merge(credit,on='title')

In [8]:
movie= movie[['movie_id','title','genres','keywords','overview','cast','crew']]

In [9]:
movie['genres'].values

array(['[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]',
       '[{"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 28, "name": "Action"}]',
       '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]',
       ...,
       '[{"id": 35, "name": "Comedy"}, {"id": 18, "name": "Drama"}, {"id": 10749, "name": "Romance"}, {"id": 10770, "name": "TV Movie"}]',
       '[]', '[{"id": 99, "name": "Documentary"}]'], dtype=object)

In [10]:
# preposing

In [11]:
movie.isnull().sum()

movie_id    0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        4
dtype: int64

In [12]:
movie.dropna(inplace=True)

In [13]:
def convert(obj):
    res=[]
    for i in ast.literal_eval(obj):
        res.append(i['name'])
    return res 

In [14]:
movie['genres'] = movie['genres'].apply(convert)
movie['keywords'] = movie['keywords'].apply(convert)

In [15]:
def convert1(obj):
    res=[]
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
            res.append(i['name'])
            count+=1
    return res 


In [16]:
movie['cast']=movie['cast'].apply(convert1)

In [17]:
def convert2(obj):
    res = []
    crew_list = ast.literal_eval(obj)  # Convert the string to a list of dictionaries    
    for i in crew_list:
        if "director" in i['job'].lower():
            res.append(i['job']) 
    return res

In [18]:
movie['overview']=movie['overview'].apply(lambda x:x.split())

In [19]:
movie['genres']=movie['genres'].apply(lambda x:[i.replace(' ', "")for i in x])
movie['keywords']=movie['keywords'].apply(lambda x:[i.replace(' ', "")for i in x])
movie['cast']=movie['cast'].apply(lambda x:[i.replace(' ', "")for i in x])
# movie['genres'].apply(lambda x:[i.replace(' ', "")for i in x])

In [20]:
movie['tags'] = movie['genres'] + movie['keywords'] +movie['cast']+ movie['overview']

In [21]:
new_df = movie[['movie_id','title','tags']]

In [22]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\tulsi\AppData\Local\Temp\ipykernel_24240\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [23]:
new_df['tags']=new_df['tags'].apply(lambda x:x .lower())

C:\Users\tulsi\AppData\Local\Temp\ipykernel_24240\1495590577.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x .lower())


# vectorization
# bag of words
text vector 

In [24]:
cv= CountVectorizer(max_features=5000,stop_words='english')

In [25]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [26]:
feature_names = cv.get_feature_names_out()

In [27]:
ps = PorterStemmer()

In [28]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y) 

In [29]:
new_df['tags'].apply(stem)

0       action adventur fantasi sciencefict culturecla...
1       adventur fantasi action ocean drugabus exotici...
2       action adventur crime spi basedonnovel secreta...
3       action crime drama thriller dccomic crimefight...
4       action adventur sciencefict basedonnovel mar m...
                              ...                        
4804    action crime thriller unitedstates–mexicobarri...
4805    comedi romanc edwardburn kerrybishé marshadiet...
4806    comedi drama romanc tvmovi date loveatfirstsig...
4807    danielhenney elizacoup billpaxton when ambiti ...
4808    documentari obsess camcord crush dreamgirl dre...
Name: tags, Length: 4802, dtype: object

In [30]:
similarity = cosine_similarity(vectors)

we go to dataset throung Index -> Similarity-> sort-> top 5 movie should be fatched

In [31]:
def recommend(movies):
    movie_index = new_df[new_df['title'] == movies].index[0]
    distance =similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:11]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [32]:
recommend("Batman Begins")

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman
Amidst the Devil's Wings
Batman v Superman: Dawn of Justice
Defendor
Mi America
Batman Returns


In [3]:
# pickle.dump(new_df,open('movies.pkl','wb'))

In [33]:
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [34]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [35]:
new_df.sample()

,movie_id,title,tags
1516,11519,1941,action comedy submarine california worldwarii ...
